# AnzoGraph DB

>[AnzoGraph DB](https://docs.cambridgesemantics.com/anzograph/v3.1/userdoc/Home.htm) is a graph database and knowledge discovery tool compliant with [RDF](https://www.w3.org/RDF/) and [SPARQL](https://www.w3.org/TR/sparql11-query/) that can be integrated with Apache Zeppelin third-party visualization application. \
> [AnzoGraph DB](https://docs.cambridgesemantics.com/anzograph/v3.1/userdoc/Home.htm) is a high performance graph OLAP database that lets users perform BI-style analytics with unparalled speed and scalability. \
> [AnzoGraph DB](https://docs.cambridgesemantics.com/anzograph/v3.1/userdoc/Home.htm) uses standards from the W3C regarding RDF data formats and the SPARQL query language. \
> [AnzoGraph DB](https://docs.cambridgesemantics.com/anzograph/v3.1/userdoc/Home.htm) can be deployed in cloud environments such as Amazon AWS, Google Cloud, Microsoft Azure, and IBM Cloud Pak, or on-premises on Linux bare metal and virtual machines. \
> [AnzoGraph DB](https://docs.cambridgesemantics.com/anzograph/v3.1/userdoc/Home.htm) also supports Docker and Kubernetes deployments with data staged locally or in shared NFS, HDFS, or object storage. 

> This notebook shows how to use LLMs to provide a natural language interface to an [AnzoGraph DB](https://docs.cambridgesemantics.com/anzograph/v3.1/userdoc/Home.htm) graph database or an ontology local file.

## Setting up

You can get a local `AnzoGraph DB` instance running via the [AnzoGraph DB Docker image](https://hub.docker.com/r/cambridgesemantics/anzograph):  

```
docker pull cambridgesemantics/anzograph
```
```
docker run -d -p 80:8080 -p 7070:7070 -p 8443:8443 -p 8256:8256 -v C:\shared-files:/opt/shared-files --name anzograph cambridgesemantics/anzograph:latest (Windows)
docker run -d -p 80:8080 -p 7070:7070 -p 8443:8443 -p 8256:8256 -v /home/username/shared-files:/opt/shared-files --name anzograph cambridgesemantics/anzograph:latest (Linux)
```

In this case, we are creating a shared volume between the local disk and the docker container. \
You just need to copy your ontology file to this local directory, and it will automatically be available for use in AnzoGraph DB.


## LangChain packages

Install

```bash
pip install langchain langchain-community langchain-openai
```

## Dependencies

Install the [rdflib](https://github.com/RDFLib/rdflib) and [SPARQLWrapper](https://github.com/RDFLib/sparqlwrapper) packages with

```bash
pip install rdflib==7.0.0
pip install SPARQLWrapper==2.0.0
```

## Endpoint Types and HTTP Methods
The table below describes the SPARQL and RDF Graph Store HTTP endpoints. Both of the endpoints can be used to send requests through the AnzoGraph DB front end or directly to the database (back end).

| Endpoint        | Description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
|-----------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| SPARQL          | The SPARQL endpoint accepts HTTP GET and POST methods. Use GET to read data from the endpoint (SELECT, ASK, CONSTRUCT, DESCRIBE queries), and use POST to update data via the endpoint (INSERT, INSERT DATA, CREATE, DELETE, DELETE DATA, DROP queries). Update queries must use the POST method, but read-only queries can be submitted using GET or POST.                                                                                                                        |
| RDF Graph Store | The RDF graph store endpoint supports create, read, update, and delete (CRUD) operations and enables programmers to work with RDF graphs in a way that is similar to REST-style interfaces. The graph store endpoint supports GET, POST, UPDATE, and DELETE HTTP methods.                                                                                                                                                                                                          |


## Endpoint Base URLs
This base URL that you use to connect to an AnzoGraph DB endpoint depends on whether you want to connect to the SPARQL endpoint or the RDF Graph Store endpoint. This section provides details about the base URLs for each endpoint:

- SPARQL Endpoint Base URL
- RDF Graph Store Endpoint Base URL

### SPARQL Endpoint Base URL
In this development, it will be employed queries on the SPARQL endpoint.
To connect to the AnzoGraph DB SPARQL endpoint, use the following base URL:

protocol://hostname:port/sparql
The table below describes each of the base URL components:


| Component | Description |
|-----------|-------------|
| protocol | The protocol to use for the connection: http for HTTP protocol or https for SSL protocol. SPARQL HTTP or HTTPS protocol can be enabled and disabled via the enable_sparql_protocol and enable_ssl_protocol settings. |
| hostname | The DNS name or IP address of the AnzoGraph DB host server. For clusters, this is the name or IP address of the leader server. |
| port | The port number for the endpoint. The port that you specify depends on the protocol and whether the request is sent to the AnzoGraph DB front end or back end. The front end requires that you use Basic Authentication to connect. The back end does not support authentication. Consider whether the client application supports authentication when specifying the port.  <br/><br/> **Front end**: If the front end ports were mapped to the default HTTP (80) and HTTPS (443) ports on the local host when the front end was deployed, do not specify a port. If the front end ports were mapped to non-default HTTP and HTTPS ports, specify the appropriate port based on the protocol. <br/> **Back end**: The port is either the HTTP sparql_protocol_port or the HTTPS ssl_protocol_port. By default, the HTTP SPARQL protocol port is 7070, and the HTTPS SSL protocol port is 8256. |
| sparql | The path for the SPARQL endpoint. |



For example, the following base URLs connect to the front end HTTP and HTTPS SPARQL endpoints. Because the ports for this deployment are mapped to the default HTTP and HTTPS ports on the local host, the port does not need to be specified in the URL (should be port :80): \
http://10.100.10.20/sparql ("10.100.10.20" is an example, usually you can use "localhost") \
https://10.100.10.20/sparql

The example URLs below connect to the back end HTTP and HTTPS SPARQL endpoints. In the examples, AnzoGraph DB is using the default SPARQL protocol and SSL protocol ports: \
http://10.100.10.20:7070/sparql \
https://10.100.10.20:8256/sparql


## Specifying the ontology


Previously, you crated a shared volume between the local disk and the docker container. \
Copy your ontology file to the shared-files local directory, and it will automatically be available for use in AnzoGraph DB. 

In order for the LLM to be able to generate SPARQL, it needs to know the knowledge graph schema (the ontology). \
It can be provided using one of two parameters on the `AnzoGraphDBGraph` class:

* `query_ontology`: a `CONSTRUCT` query that is executed on the SPARQL backend endpoint (port :7070 for http, or :8256 for https) and returns the KG schema statements, building a RDF 'subgraph' from the database's graph. We recommend that you store the ontology in its own named graph, which will make it easier to get only the relevant statements (as the example below). `DESCRIBE` queries are not supported, because `DESCRIBE` returns the Symmetric Concise Bounded Description (SCBD), i.e. also the incoming class links.
* `source_file`: a local source RDF ontology file. Supported RDF formats are `Turtle`, `N-Triple`, `N-Quad`, `TriG ` or `JSON-LD` files. Turtle (.ttl) files are the most usual.



## Download the FOAF ontology and load it into AnzoGraph DB
For this example code, download from the file for the FOAF ontology. 

Please download it from the SPAR Ontologies repository (learn more at http://www.sparontologies.net): \
https://github.com/SPAROntologies/foaf/blob/master/docs/current/foaf.ttl

Then:

* Copy the Turtle (.ttl) ontology file into the shared-files folder in your disk (the bind folder created through docker run)
* Start AnzoGraph DB frontend with the following URI: "http://localhost:80"
* Username: admin
* Password: Passw0rd1

#### Load the ontology file using the query window of the graphic user interface. Run the following SPARQL query:
```
LOAD WITH 'global' <dir:/opt/shared-files/foaf.ttl> INTO GRAPH <http://anzograph.com/foaf>
```

## Dependencies install

In [ ]:
%pip install --quiet rdflib==7.0.0
%pip install --quiet SPARQLWrapper==2.0.0

## Creating the Graph object
There are two methods to create the graph using the `AnzoGraphDBGraph` integration in LangChain

### 1. Querying the loaded ontology against AnzoGraph DB

In [3]:
from langchain_community.graphs.anzograph_graph import AnzoGraphDBGraph

# Feeding the graph schema using a CONSTRUCT query against the AnzoGraph DB backend endpoint

graph = AnzoGraphDBGraph(
    query_endpoint="http://localhost:7070/sparql",  # Or "https://localhost:8256/sparql"
    query_ontology="CONSTRUCT {?s ?p ?o} FROM <http://anzograph.com/foaf> WHERE {?s ?p ?o}",
)

### 2. Loading the ontology directly from a local source file

In [1]:
from langchain_community.graphs.anzograph_graph import AnzoGraphDBGraph

# Feeding the graph schema using a local RDF file source

graph = AnzoGraphDBGraph(
    query_endpoint="http://example.com/sparql",  # Placeholder URL string; not used when operating with a local source_file
    source_file="/path/to/langchain_graphdb_tutorial/foaf.ttl",  # change the path here
    standard="rdf",
    local_copy="test.ttl",
)

### Checking the ontology via the graph's content

In [2]:
# This code snippet is for assessment purposes only

print(len(graph.graph))

graph.load_schema()
print(graph.get_schema)

282
In the following, each IRI is followed by the local name and optionally its description in parentheses. 
The RDF graph supports the following node types:
<http://www.w3.org/2000/01/rdf-schema#Class> (Class, None), <http://www.w3.org/2002/07/owl#Class> (Class, None), <http://www.w3.org/2002/07/owl#AnnotationProperty> (AnnotationProperty, None), <http://www.w3.org/2002/07/owl#Ontology> (Ontology, None), <http://www.w3.org/2002/07/owl#DatatypeProperty> (DatatypeProperty, None), <http://www.w3.org/2002/07/owl#FunctionalProperty> (FunctionalProperty, None), <http://www.w3.org/2002/07/owl#ObjectProperty> (ObjectProperty, None), <http://www.w3.org/2002/07/owl#InverseFunctionalProperty> (InverseFunctionalProperty, None)
The RDF graph supports the following relationships:
<http://www.w3.org/2000/01/rdf-schema#range> (range, None), <http://purl.org/dc/terms/date> (date, None), <http://www.w3.org/2003/06/sw-vocab-status/nsterm_status> (nsterm_status, None), <http://www.w3.org/2000/01/rdf-sche

Either way, the ontology (schema) is fed to the LLM as `Turtle` since `Turtle` with appropriate prefixes is most compact and easiest for the LLM to remember.

The Turtle serialized graph looks the following:

```
@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix ns1: <http://www.w3.org/2003/06/sw-vocab-status/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

foaf:Agent a rdfs:Class,
        owl:Class ;
    rdfs:label "(foaf) Agent" ;
    rdfs:comment "An agent (eg. person, group, software or physical artifact)." ;
    rdfs:isDefinedBy foaf: ;
    owl:disjointWith foaf:Document,
        foaf:OnlineAccount,
        foaf:Project ;
    ns1:nsterm_status "unstable" .

foaf:Document a rdfs:Class,
        owl:Class ;
    rdfs:label "(foaf) Document" ;
    rdfs:comment "A document." ;
    rdfs:isDefinedBy foaf: ;
    owl:disjointWith foaf:Agent,
        foaf:OnlineAccount,
        foaf:Project ;
    ns1:nsterm_status "testing" .

foaf:Project a rdfs:Class,
        owl:Class ;
    rdfs:label "(foaf) Project" ;
    rdfs:comment "A project (a collective endeavour of some kind)." ;
    rdfs:isDefinedBy foaf: ;
    owl:disjointWith foaf:Agent,
        foaf:Document,
        foaf:OnlineAccount ;
    ns1:nsterm_status "unstable" .

    ...

 ```


## Question Answering against the FOAF dataset

#### We can now use the `AnzoGraphDBQAChain` query the graph database or local file.

Any available OpenAI model can be used here.\
We used 'gpt-4o', 'gpt-4-turbo', and 'gpt-4-turbo-preview', considering their novelty and position in the LMSys Leaderboard.\
The 'gpt-4-turbo-preview' model was initially considered the most performant.\
Consult with the OpenAI API https://platform.openai.com/docs/models for the most up-to-date models at the time of reading.

In [3]:
import os

from langchain_community.chains.graph_qa.anzograph import AnzoGraphDBQAChain
from langchain_openai import ChatOpenAI

# We'll be using an OpenAI model which requires an OpenAI API Key.
# However, other models are available as well:
# https://python.langchain.com/docs/integrations/chat/

# Set the environment variable `OPENAI_API_KEY` to your OpenAI API key
os.environ["ORGANIZATION_ID"] = "org-***"  # (in case of an organization's account)
os.environ["OPENAI_API_KEY"] = "sk-***"


# Any available OpenAI model can be used here.

llm = ChatOpenAI(
    model="gpt-4-turbo",
    api_key=os.environ["OPENAI_API_KEY"],
    openai_organization=os.environ["ORGANIZATION_ID"],  # if that is the case
    temperature=0,
)

# Intantiate the chain with the LLM and the graph
chain = AnzoGraphDBQAChain.from_llm(
    llm=llm,
    graph=graph,
    verbose=True,
)

Let's perform a query against the FOAF ontology.

In [6]:
from langchain_community.chains.graph_qa.anzograph import AnzoGraphDBQAChain

# Instantiate the chain connecting the LLM and the graph, enabling verbose output for detailed debugging

chain = AnzoGraphDBQAChain.from_llm(
    llm=llm,
    graph=graph,
    verbose=True,
)

In [4]:
chain.invoke({chain.input_key: "Find entities and list them."})[chain.output_key]



> Entering new AnzoGraphDBQAChain chain...
Identified intent:
SELECT
Generated SPARQL:
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

SELECT ?entity
WHERE {
    ?entity rdf:type ?type .
    FILTER (?type IN (owl:Class, rdfs:Class, owl:AnnotationProperty, owl:Ontology, owl:DatatypeProperty, owl:FunctionalProperty, owl:ObjectProperty, owl:InverseFunctionalProperty))
}
Full Context:
[(rdflib.term.URIRef('http://xmlns.com/foaf/0.1/Agent'),), (rdflib.term.URIRef('http://xmlns.com/foaf/0.1/Document'),), (rdflib.term.URIRef('http://xmlns.com/foaf/0.1/Project'),), (rdflib.term.URIRef('http://xmlns.com/foaf/0.1/Agent'),), (rdflib.term.URIRef('http://xmlns.com/foaf/0.1/Document'),), (rdflib.term.URIRef('http://xmlns.com/foaf/0.1/Project'),), (rdflib.term.BNode('nf5f7add225254cb4bfb798306626d28db1'),), (rdflib.term.BNode('nf5f7add225254cb4bfb798306626d28db2'),), (rdflib.term.BNode('nf5f

'The entities listed in the SPARQL query results include various types and properties from the FOAF (Friend of a Friend) ontology and Dublin Core terms. Here are the entities identified:\n\n1. **Types of Entities:**\n   - Agent\n   - Document\n   - Project\n   - PersonalProfileDocument\n   - Organization\n   - Group\n   - OnlineAccount\n   - Image\n   - Person\n\n2. **Properties and Terms:**\n   - contributor\n   - creator\n   - date\n   - format\n   - language\n   - publisher\n   - title\n   - nsterm_status\n   - aimChatID\n   - birthday\n   - family_name\n   - firstName\n   - geekcode\n   - gender\n   - givenname\n   - jabberID\n   - myersBriggs\n   - name\n   - plan\n   - sha1\n   - surname\n   - nick\n   - primaryTopic\n   - currentProject\n   - fundedBy\n   - homepage\n   - img\n   - interest\n   - knows\n   - logo\n   - mbox\n   - member\n   - pastProject\n   - phone\n   - publications\n   - schoolHomepage\n   - thumbnail\n   - tipjar\n   - topic_interest\n   - weblog\n   - workI

Or:

In [ ]:
chain.invoke(
    {chain.input_key: "Enumerate all Classes (rdfs and owl) in FOAF ontology."}
)[chain.output_key]



> Entering new AnzoGraphDBQAChain chain...
Identified intent:
SELECT
Generated SPARQL:
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

SELECT DISTINCT ?class
WHERE {
    { ?class a rdfs:Class . FILTER (STRSTARTS(STR(?class), STR(foaf:))) }
    UNION
    { ?class a owl:Class . FILTER (STRSTARTS(STR(?class), STR(foaf:))) }
}
Full Context:
[(rdflib.term.URIRef('http://xmlns.com/foaf/0.1/Agent'),), (rdflib.term.URIRef('http://xmlns.com/foaf/0.1/Document'),), (rdflib.term.URIRef('http://xmlns.com/foaf/0.1/Project'),), (rdflib.term.URIRef('http://xmlns.com/foaf/0.1/PersonalProfileDocument'),), (rdflib.term.URIRef('http://xmlns.com/foaf/0.1/Organization'),), (rdflib.term.URIRef('http://xmlns.com/foaf/0.1/Group'),), (rdflib.term.URIRef('http://xmlns.com/foaf/0.1/OnlineAccount'),), (rdflib.term.URIRef('http://xmlns.com/foaf/0.1/Image'),), (rdflib.term.URIRef('http://xmlns.com/foaf/0.1/Person'),)]

> F

'The classes in the FOAF (Friend of a Friend) ontology include Agent, Document, Project, PersonalProfileDocument, Organization, Group, OnlineAccount, Image, and Person. These classes are used to describe various types of entities and relationships in social networks and personal data.'

And a little more complex one.

In [9]:
chain.invoke(
    {chain.input_key: "Can you bring relations among the recognized entities?"}
)[chain.output_key]



> Entering new AnzoGraphDBQAChain chain...
Identified intent:
SELECT
Generated SPARQL:
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

SELECT ?entity ?relation ?relatedEntity
WHERE {
    ?entity ?relation ?relatedEntity .
    FILTER (?relation IN (rdf:type, rdfs:domain, owl:disjointWith, rdf:rest, owl:unionOf, owl:complementOf, rdf:first, rdfs:subPropertyOf, rdfs:isDefinedBy, rdfs:comment, rdfs:range, rdfs:label, owl:inverseOf, rdfs:subClassOf, owl:priorVersion))
}
Full Context:
[(rdflib.term.URIRef('http://xmlns.com/foaf/0.1/depiction'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/2002/07/owl#ObjectProperty')), (rdflib.term.URIRef('http://xmlns.com/foaf/0.1/publications'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#domain'), rdflib.term.URIRef('http://xmlns.com/foaf/0.1/Person')), (rdflib.term.URIR

'Certainly! Here\'s an overview of the relationships among the recognized entities based on the provided RDF data:\n\n1. **Classes and Their Properties**:\n   - `foaf:Person` is defined as a subclass of `foaf:Agent` and has various properties such as `foaf:firstName`, `foaf:surname`, `foaf:family_name`, `foaf:givenname`, `foaf:mbox`, `foaf:homepage`, and `foaf:gender`. It is disjoint with `foaf:Organization` and `foaf:Group`.\n   - `foaf:Organization` is also a subclass of `foaf:Agent` and is defined to be disjoint with `foaf:Person` and `foaf:Group`.\n   - `foaf:Group` is defined as a class of agents, subclass of `foaf:Agent`, and is disjoint with `foaf:Person` and `foaf:Organization`.\n   - `foaf:Document` and `foaf:Image` are subclasses of each other, with specific properties like `foaf:depiction` for images. `foaf:Document` is disjoint with `foaf:Agent`, `foaf:OnlineAccount`, and `foaf:Project`.\n   - `foaf:Project` and `foaf:OnlineAccount` are defined as classes and have specific 

## Chain modifiers

AnzoGraph DB QA Chain allows prompt refinement for further improvement of the QA chains and enhancing the overall user experience, in order to prevent errors in LangChain's responses to user prompts, caused by formatting issues in SPARQL queries generated by the LLM. Generation Templates were developed for the SELECT and UPDATE query intents to mitigate this risk of response errors in the Python environment, especially instructing the LLM to avoid the use of backticks.


### "SPARQL Generation" prompt

The prompt is used for the SPARQL query generation (SELECT or UPDATE statements) based on the user question detected intent, and the KG schema.

- `sparql_generation_prompt`

    Default value:
  ````python
    ANZOGRAPHDB_INTENT_PROMPT = PromptTemplate(
    input_variables=["prompt"], template=ANZOGRAPHDB_INTENT_TEMPLATE)

    ANZOGRAPHDB_GENERATION_SELECT_TEMPLATE = """Task: Generate a SPARQL SELECT statement for querying a graph database.
    Do NOT use neither backticks (```) nor backsticks with "sparql" (```sparql) in the Generated SPARQL. 
    For instance, to find all email addresses of John Doe, the following query would be suitable:

    PREFIX foaf: <http://xmlns.com/foaf/0.1/>
    SELECT ?email
    WHERE {{
        ?person foaf:name "John Doe" .
        ?person foaf:mbox ?email .
    }}

    Instructions:
    Use only the node types and properties provided in the schema.
    Do not use any node types and properties that are not explicitly provided.
    Include all necessary prefixes.
    Schema:
    {schema}
    Note: Be as concise as possible.
    Do not include any explanations or apologies in your responses.
    Do not respond to any questions that ask for anything else than for you to construct a SPARQL query.
    Do not include any text except the SPARQL query generated.

    The question is:
    {prompt}"""
    ANZOGRAPHDB_GENERATION_SELECT_PROMPT = PromptTemplate(
        input_variables=["schema", "prompt"], template=ANZOGRAPHDB_GENERATION_SELECT_TEMPLATE
    )

    ANZOGRAPHDB_GENERATION_UPDATE_TEMPLATE = """Task: Generate a SPARQL UPDATE statement for updating a graph database.
    Do NOT use neither backticks (```) nor backsticks with "sparql" (```sparql) in the Generated SPARQL. 
    For instance, to add 'jane.doe@foo.bar' as a new email address for Jane Doe, the following query would be suitable:

    PREFIX foaf: <http://xmlns.com/foaf/0.1/>
    INSERT {{
        ?person foaf:mbox <mailto:jane.doe@foo.bar> .
    }}
    WHERE {{
        ?person foaf:name "Jane Doe" .
    }}

    Instructions:
    Make the query as short as possible and avoid adding unnecessary triples.
    Use only the node types and properties provided in the schema.
    Do not use any node types and properties that are not explicitly provided.
    Include all necessary prefixes.
    Schema:
    {schema}
    Note: Be as concise as possible.
    Do not include any explanations or apologies in your responses.
    Do not respond to any questions that ask for anything else than for you to construct a SPARQL query.
    Return only the generated SPARQL query, nothing else.

    The information to be inserted is:
    {prompt}"""
    ANZOGRAPHDB_GENERATION_UPDATE_PROMPT = PromptTemplate(
        input_variables=["schema", "prompt"], template=ANZOGRAPHDB_GENERATION_UPDATE_TEMPLATE
    )

  ````

### "SPARQL Fix" prompt

In our tests, the use of the SPARQL Generation prompts was enough totally prevent errors or missing prefixes, etc. In case it is not sufficient with the use of any LLM, a SPARQL fix prompt was also created and it is available in the documentation of the chain prompts templates. The chain will try to amend this by prompting the LLM to correct it a certain number of times.

- `sparql_fix_prompt`

    Default value:
  ````python
    ANZOGRAPHDB_QA_FIX_TEMPLATE = """
    This following SPARQL query delimited by triple backticks
    ```
    {generated_sparql}
    ```
    is not valid.
    The error delimited by triple backticks is
    ```
    {error_message}
    ```

    Give me a correct version of the SPARQL query.
    Include  the SPARQL query generated.
    Do not change the logic of the query.
    Do not include any explanations or apologies in your responses.
    Do not wrap the query in backticks.
    Do not include any text except the SPARQL query generated.
    The ontology schema delimited by triple backticks in Turtle format is:
    ```
    {schema}
    ```
    """
    ANZOGRAPHDB_QA_FIX_PROMPT = PromptTemplate(
        input_variables=["error_message", "generated_sparql", "schema"],
        template=ANZOGRAPHDB_QA_FIX_TEMPLATE
    )

  ````

- `max_fix_retries`
  
    Default value: `5`

### "Answering" prompt

The prompt is used for answering the question based on the results returned from the database and the initial user question. By default, the LLM is instructed to only use the information from the returned result(s). If the result set is empty, the LLM should inform that it can't answer the question.

- `qa_prompt`
  
  Default value:
  ````python
    ANZOGRAPHDB_QA_TEMPLATE = """Task: Generate a natural language response from the results of a SPARQL query.
    You are an assistant that creates well-written and human understandable answers.
    The information part contains the information provided, which you can use to construct an answer.
    The information provided is authoritative, you must never doubt it or try to use your internal knowledge to correct it.
    Make your response sound like the information is coming from an AI assistant, but don't add any information.
    Information:
    {context}

    Question: {prompt}
    Helpful Answer:"""
    ANZOGRAPHDB_QA_PROMPT = PromptTemplate(
        input_variables=["context", "prompt"], template=ANZOGRAPHDB_QA_TEMPLATE)
  ````

<hr>